<a href="https://colab.research.google.com/github/MorningStarTM/movie-genre-classification/blob/main/movie_genre_classification-v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import libraries
import pandas as pd
import numpy as np
import re
import nltk
import tensorflow as tf
from keras.utils import to_categorical
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, LSTM, ReLU, Flatten, Embedding, SpatialDropout1D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
#reading data
file = open("/content/drive/MyDrive/DataSet/csv dataset/train_data.txt", encoding='utf-8', errors='ignore').read().split('\n')

#size of data
len(file)

54215

# Creating Dataset

In [4]:
#initialize the dataframe
df = pd.DataFrame()

In [5]:
def divide_data(texts):
    genre = []
    desc = []
    for text in texts:
        if len(text.split(':::')) == 4:
            splited = text.split(":::")
            genre.append(splited[-2])
            desc.append(splited[-1])
    return genre, desc

In [6]:
genre, desc = divide_data(file)

In [7]:
#size of data
len(genre), len(desc)

(54214, 54214)

In [8]:
#creating dataframe for data
df['genre'] = genre
df['desc'] = desc

In [9]:
df.head()

,genre,desc
0,drama,Listening in to a conversation between his do...
1,thriller,A brother and sister with a past incestuous r...
2,adult,As the bus empties the students for their fie...
3,drama,To help their unemployed father make ends mee...
4,drama,The film's title refers not only to the un-re...


In [10]:
df['desc'][0]

' Listening in to a conversation between his doctor and parents, 10-year-old Oscar learns what nobody has the courage to tell him. He only has a few weeks to live. Furious, he refuses to speak to anyone except straight-talking Rose, the lady in pink he meets on the hospital stairs. As Christmas approaches, Rose uses her fantastical experiences as a professional wrestler, her imagination, wit and charm to allow Oscar to live life and love to the full, in the company of his friends Pop Corn, Einstein, Bacon and childhood sweetheart Peggy Blue.'

# Data Preprocessing

In [11]:
#initialize the stemmer and lemmentizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [12]:
#filtering the text
def full_form(text):
    text = text.lower()
    plain = re.sub(r'[-<>?\.,!"(\)\/[\]]', ' ', text)
    plain = plain.replace("don't", "do not")
    plain = plain.replace("won't", "will not")
    plain = plain.replace("haven't", "have not")
    plain = plain.replace("can't", "cannot")
    plain = plain.replace("she's", "she is")
    plain = plain.replace("he's", "he is")
    plain = plain.replace("there're", "there are")
    plain = plain.replace("they'd", "they would")
    plain = plain.replace("\'ll", " will")
    return plain   

In [13]:
#preprocessing  text 
def remove_stop_words(data):
  corpus = []
  for i in range(0, len(data)):
    #split the sentence
    plain = full_form(data[i])
    sentence = plain.split()
    
    #check and stem the word
    review_processed = [stemmer.stem(word) for word in sentence if not word in stopwords.words('english')]
    #rebuild the sentence
    review_joint = ' '.join(review_processed)
    #add the sentence into list
    corpus.append(review_joint)
  return corpus

In [14]:
processed_desc = remove_stop_words(df['desc'])

In [15]:
#sample length for getting overall idea for max length of sentence
length = [len(processed_desc[i].split()) for i in range(40)]
length

[54,
 15,
 56,
 109,
 58,
 170,
 33,
 31,
 42,
 43,
 37,
 18,
 73,
 29,
 47,
 26,
 33,
 48,
 206,
 28,
 35,
 19,
 88,
 46,
 28,
 30,
 56,
 28,
 34,
 74,
 43,
 63,
 65,
 18,
 57,
 41,
 63,
 41,
 24,
 57]

In [16]:
#filtering the genre
def filter_text(text):
    text = text.lower()
    plain = re.sub(r'[-<>?\.,@#$%&*!"(\)\/[\]]', '', text)
    return plain

In [17]:
filtered_genre = []
for gnre in df['genre']:
    filtered_genre.append(filter_text(gnre))

In [18]:
len(filtered_genre), len(genre)

(54214, 54214)

In [19]:
df['preprocessed_desc'] = processed_desc
df['preprocessed_genre'] = filtered_genre

In [20]:
df['label'], _ = pd.factorize(df['preprocessed_genre'])

In [21]:
df.head()

,genre,desc,preprocessed_desc,preprocessed_genre,label
0,drama,Listening in to a conversation between his do...,listen convers doctor parent 10 year old oscar...,drama,0
1,thriller,A brother and sister with a past incestuous r...,brother sister past incestu relationship curre...,thriller,1
2,adult,As the bus empties the students for their fie...,bu empti student field trip museum natur histo...,adult,2
3,drama,To help their unemployed father make ends mee...,help unemploy father make end meet edith twin ...,drama,0
4,drama,The film's title refers not only to the un-re...,film' titl refer un recov bodi ground zero als...,drama,0


In [22]:
len(df['label'].unique())

27

In [23]:
df['preprocessed_genre'].unique(), len(df['preprocessed_genre'].unique())

(array([' drama ', ' thriller ', ' adult ', ' documentary ', ' comedy ',
        ' crime ', ' realitytv ', ' horror ', ' sport ', ' animation ',
        ' action ', ' fantasy ', ' short ', ' scifi ', ' music ',
        ' adventure ', ' talkshow ', ' western ', ' family ', ' mystery ',
        ' history ', ' news ', ' biography ', ' romance ', ' gameshow ',
        ' musical ', ' war '], dtype=object), 27)

# Embedding

In [24]:
#sentence length
sent_length = 250
#vocabulary size
voc_size = 10000

In [25]:
#convert into one hot vector
onehot_text = [one_hot(word, voc_size) for word in df['preprocessed_desc']]

In [26]:
len(onehot_text[0]), len(df['preprocessed_desc'][0].split())

(54, 54)

In [27]:
#embedding
embedd_docs = pad_sequences(onehot_text, padding='pre', maxlen=sent_length)

In [28]:
embedd_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [29]:
y = df['label'].values

In [30]:
y_label = to_categorical(y)

In [31]:
#shape of data
len(embedd_docs), len(y_label)

(54214, 54214)

In [32]:
#convert into numpy array
X_data = np.array(embedd_docs)
y_data = np.array(y_label)

In [33]:
X_data.shape, y_data.shape

((54214, 250), (54214, 27))

In [34]:
#split dataset for training and testing
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3)

In [35]:
print(f'training: {X_train.shape} - {y_train.shape}')
print(f'testing: {X_test.shape} - {y_test.shape}')

training: (37949, 250) - (37949, 27)
testing: (16265, 250) - (16265, 27)


In [36]:
print(y_train[0])
print(len(y_train[0]))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
27


# Model

In [37]:
# Creating model
embedding_vector_features=100
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(27,activation='sigmoid'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 100)          1000000   
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 27)                2727      
                                                                 
Total params: 1,083,127
Trainable params: 1,083,127
Non-trainable params: 0
_________________________________________________________________
None


In [38]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
1186/1186 [==============================] - 255s 213ms/step - loss: 2.0250 - accuracy: 0.4157 - val_loss: 1.7754 - val_accuracy: 0.4904
Epoch 2/10
1186/1186 [==============================] - 262s 221ms/step - loss: 1.5802 - accuracy: 0.5439 - val_loss: 1.6525 - val_accuracy: 0.5150
Epoch 3/10
1186/1186 [==============================] - 259s 218ms/step - loss: 1.3305 - accuracy: 0.6149 - val_loss: 1.6332 - val_accuracy: 0.5290
Epoch 4/10
  63/1186 [>.............................] - ETA: 3:36 - loss: 1.0460 - accuracy: 0.7034

KeyboardInterrupt: ignored

In [39]:
pred = (model.predict(X_test) > 0.5).astype("int32")

509/509 [==============================] - 26s 50ms/step


In [41]:
X_test[0], y_test[0]

(array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [50]:
sample = X_test[0].reshape(1, 250)
sample.shape

(1, 250)

In [55]:
re = (model.predict(sample) > 0.9).astype("int32")

1/1 [==============================] - 0s 45ms/step


In [56]:
re

array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0]], dtype=int32)

In [57]:
model.evaluate(X_test, y_test)

509/509 [==============================] - 27s 52ms/step - loss: 1.6521 - accuracy: 0.5321


[1.652130126953125, 0.5320627093315125]